In [ ]:
!pip install streamlit pyngrok pandas -q

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("34mnxhOy5H2Kq0wBzyfZzoZs1S5_nrrFTEWa8LseCHvmXwsg")


In [ ]:
%%writefile app.py
import pandas as pd
import streamlit as st
from google.colab import userdata

# --- Define Q&A logic ---
def answer_query(query, data):
    query = query.lower()

    if "highest" in query and "production" in query:
        year = ''.join(filter(str.isdigit, query))
        if year:
            df = data[data['Year'].astype(str).str.contains(year)]
            if not df.empty:
                top_state = df.groupby('State')['Production'].sum().idxmax()
                value = df.groupby('State')['Production'].sum().max()
                return f"In {year}, {top_state} had the highest rice production of {value:,.0f} tonnes. (Source: Agriculture Dataset)"
            else:
                return f"No data available for the year {year}."

    elif "average rainfall" in query:
        for state in data['State'].unique():
            if state.lower() in query:
                avg_rain = data[data['State'].str.lower() == state.lower()]['ANNUAL'].mean()
                return f"The average annual rainfall in {state} is {avg_rain:.2f} mm. (Source: IMD Rainfall Dataset)"

    elif "top" in query and "production" in query:
        top_n = [int(s) for s in query.split() if s.isdigit()]
        n = top_n[0] if top_n else 5
        top_states = data.groupby('State')['Production'].sum().nlargest(n)
        result = "\n".join([f"{i+1}. {state}: {prod:,.0f} tonnes"
                            for i, (state, prod) in enumerate(top_states.items())])
        return f"Top {n} states with highest rice production:\n{result}\n(Source: Agriculture Dataset)"
    else:
        return "Sorry, I couldn’t understand the question clearly. Try rephrasing!"

# --- Streamlit UI ---
st.title("🌾 India Agriculture & Rainfall Q&A System")
st.write("Ask questions about rainfall and crop production across Indian states. Data sourced from public datasets (Kaggle / data.gov.in equivalent).")

# Load data
data = pd.read_csv("/content/cleaned_rainfall_crop.csv")

query = st.text_input("Ask a question (e.g. 'Which state had the highest rice production in 2010?')")
if query:
    answer = answer_query(query, data)
    st.success(answer)

Overwriting app.py


In [ ]:
import pandas as pd
data = pd.read_csv("/content/cleaned_rainfall_crop.csv")

In [ ]:
# --- Define Q&A logic ---
def answer_query(query, data):
    query = query.lower()

    if "highest" in query and "production" in query:
        year = ''.join(filter(str.isdigit, query))
        if year:
            df = data[data['Year'].astype(str).str.contains(year)]
            if not df.empty:
                top_state = df.groupby('State')['Production'].sum().idxmax()
                value = df.groupby('State')['Production'].sum().max()
                return f"In {year}, {top_state} had the highest rice production of {value:,.0f} tonnes. (Source: Agriculture Dataset)"
            else:
                return f"No data available for the year {year}."

    elif "average rainfall" in query:
        for state in data['State'].unique():
            if state.lower() in query:
                avg_rain = data[data['State'].str.lower() == state.lower()]['ANNUAL'].mean()
                return f"The average annual rainfall in {state} is {avg_rain:.2f} mm. (Source: IMD Rainfall Dataset)"

    elif "top" in query and "production" in query:
        top_n = [int(s) for s in query.split() if s.isdigit()]
        n = top_n[0] if top_n else 5
        top_states = data.groupby('State')['Production'].sum().nlargest(n)
        result = "\n".join([f"{i+1}. {state}: {prod:,.0f} tonnes"
                            for i, (state, prod) in enumerate(top_states.items())])
        return f"Top {n} states with highest rice production:\n{result}\n(Source: Agriculture Dataset)"
    else:
        return "Sorry, I couldn’t understand the question clearly. Try rephrasing!"

In [ ]:
print("🚀 Starting Streamlit app...")

# Run streamlit in background
!streamlit run app.py &>/content/logs.txt &

# Create tunnel
public_url = ngrok.connect(8501)
print("🌐 Streamlit app is live here:\n", public_url)

🚀 Starting Streamlit app...
🌐 Streamlit app is live here:
 NgrokTunnel: "https://noisily-nonconjecturable-eleonor.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &>/content/logs.txt &
public_url = ngrok.connect(8501)
print(public_url)


NgrokTunnel: "https://noisily-nonconjecturable-eleonor.ngrok-free.dev" -> "http://localhost:8501"
